In [8]:
import re

import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import missingno as msno
import pandas as pd

%matplotlib inline

plt.style.use("seaborn-v0_8")

In [2]:
prompts_train = pd.read_csv('../data/raw/prompts_train.csv')
prompts_test = pd.read_csv('../data/raw/prompts_test.csv')
sample_submission = pd.read_csv('../data/raw/sample_submission.csv')
summaries_train = pd.read_csv('../data/raw/summaries_train.csv')
summaries_test = pd.read_csv('../data/raw/summaries_test.csv')

In [3]:
train = pd.merge(prompts_train, summaries_train, on='prompt_id', how='right')

In [4]:
train

,prompt_id,prompt_question,prompt_title,prompt_text,student_id,text,content,wording
0,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,000e8c3c7ddb,The third wave was an experimentto see how peo...,0.205683,0.380538
1,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...",0020ae56ffbf,They would rub it up with soda to make the sme...,-0.548304,0.506755
2,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,004e978e639e,"In Egypt, there were many occupations and soci...",3.128928,4.231226
3,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,005ab0199905,The highest class was Pharaohs these people we...,-0.210614,-0.471415
4,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,0070c9e7af47,The Third Wave developed rapidly because the ...,3.272894,3.219757
...,...,...,...,...,...,...,...,...
7160,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...",ff7c7e70df07,They used all sorts of chemical concoctions to...,0.205683,0.380538
7161,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,ffc34d056498,The lowest classes are slaves and farmers slav...,-0.308448,0.048171
7162,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,ffd1576d2e1b,they sorta made people start workin...,-1.408180,-0.493603
7163,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,ffe4a98093b2,An ideal tragety has three elements that make ...,-0.393310,0.627128


In [7]:
train.iloc[0, :]

prompt_id                                                     814d6b
prompt_question    Summarize how the Third Wave developed over su...
prompt_title                                          The Third Wave
prompt_text        Background \r\nThe Third Wave experiment took ...
student_id                                              000e8c3c7ddb
text               The third wave was an experimentto see how peo...
content                                                     0.205683
wording                                                     0.380538
Name: 0, dtype: object

In [32]:
def word_overlap_count(row):
    STOP_WORDS = set(stopwords.words('english'))  # type: ignore 
    def check_is_stop_word(word):
        return word in STOP_WORDS
    
    prompt_words = row['prompt_text']
    summary_words = row['text']
    if STOP_WORDS:
        prompt_words = list(filter(check_is_stop_word, prompt_words))
        summary_words = list(filter(check_is_stop_word, summary_words))
    return len(set(prompt_words).intersection(set(summary_words)))


def ngram_co_occurrence(row, n: int):
    def ngrams(token, n):
        # Use the zip function to help us generate n-grams
        # Concatentate the tokens into ngrams and return
        ngrams = zip(*[token[i:] for i in range(n)])
        return [" ".join(ngram) for ngram in ngrams]

    # Tokenize the original text and summary into words
    original_tokens = row['prompt_text'].split(" ")
    summary_tokens = row['text'].split(" ")

    # Generate n-grams for the original text and summary
    original_ngrams = set(ngrams(original_tokens, n))
    summary_ngrams = set(ngrams(summary_tokens, n))

    # Calculate the number of common n-grams
    common_ngrams = original_ngrams.intersection(summary_ngrams)


    return len(common_ngrams)

In [33]:
word_overlap_count(train.iloc[0, :])

8

In [34]:
ngram_co_occurrence(train.iloc[0, :], n=2), ngram_co_occurrence(train.iloc[0, :], n=3)

(2, 0)